# Étape 1: Télécharger et explorer les données

In [1]:
# Installer la bibliothèque datasets si nécessaire
# !pip install datasets

from datasets import load_dataset
import matplotlib.pyplot as plt

In [2]:
# Charger les données MMDocIR
mmdocir_data = load_dataset("MMDocIR/MMDocIR-Challenge")
print("MMDocIR-Challenge :")
print(mmdocir_data)

README.md:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

MMDocIR_doc_passages.parquet:   0%|          | 0.00/1.60G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20395 [00:00<?, ? examples/s]

MMDocIR-Challenge :
DatasetDict({
    train: Dataset({
        features: ['doc_name', 'domain', 'passage_id', 'image_path', 'image_binary', 'ocr_text', 'vlm_text'],
        num_rows: 20395
    })
})


In [3]:
# Charger les données M2KR avec la configuration 'challenge_data'
m2kr_data = load_dataset("Jingbiao/M2KR-Challenge", "challenge_data")
print("\nM2KR-Challenge (challenge_data) :")
print(m2kr_data)

README.md:   0%|          | 0.00/4.83k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/169k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6415 [00:00<?, ? examples/s]


M2KR-Challenge (challenge_data) :
DatasetDict({
    train: Dataset({
        features: ['pos_item_ids', 'pos_item_contents', 'question', 'question_id', 'instruction', 'img_path'],
        num_rows: 6415
    })
})


In [4]:
# Charger les données M2KR avec la configuration 'challenge_passage'
m2kr_passages = load_dataset("Jingbiao/M2KR-Challenge", "challenge_passage")
print("\nM2KR-Challenge (challenge_passage) :")
print(m2kr_passages)

train-00000-of-00001.parquet:   0%|          | 0.00/24.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/47318 [00:00<?, ? examples/s]


M2KR-Challenge (challenge_passage) :
DatasetDict({
    train: Dataset({
        features: ['passage_id', 'passage_content', 'page_screenshot'],
        num_rows: 47318
    })
})


In [5]:

# # Explorer un échantillon de MMDocIR
# print("\nExemple de données MMDocIR :")
# print(mmdocir_data["train"][0])

# # Explorer un échantillon de M2KR (challenge_data)
# print("\nExemple de données M2KR (challenge_data) :")
# print(m2kr_data["train"][0])

# # Explorer un échantillon de M2KR (challenge_passage)
# print("\nExemple de données M2KR (challenge_passage) :")
# print(m2kr_passages["train"][0])

In [6]:
# Compter le nombre de documents dans MMDocIR
num_docs_mmdocir = len(mmdocir_data["train"])
print(f"\nNombre de documents dans MMDocIR : {num_docs_mmdocir}")

# Compter le nombre de documents dans M2KR (challenge_data)
num_docs_m2kr = len(m2kr_data["train"])
print(f"Nombre de documents dans M2KR (challenge_data) : {num_docs_m2kr}")

# Compter le nombre de passages dans M2KR (challenge_passage)
num_passages_m2kr = len(m2kr_passages["train"])
print(f"Nombre de passages dans M2KR (challenge_passage) : {num_passages_m2kr}")


Nombre de documents dans MMDocIR : 20395
Nombre de documents dans M2KR (challenge_data) : 6415
Nombre de passages dans M2KR (challenge_passage) : 47318


In [7]:
# Afficher le texte OCR d'un document MMDocIR
print("\nTexte OCR d'un document MMDocIR :")
print(mmdocir_data["train"][0]["ocr_text"][:500])  # Affiche les 500 premiers caractères


Texte OCR d'un document MMDocIR :
Towards Verifiable Generation: A Benchmark for Knowledge-aware
Language Model Attribution

Xinze Li’, Yixin Cao’', Liangming Pan*, Yubo Ma’, Aixin Sun'*

1 §-Lab, Nanyang Technological University
? Singapore Management University * University of California, Santa Barbara

{xinze002,
yxcao@smu.edu.sg

Abstract

Although achieving great success, Large Lan-
guage Models (LLMs) usually suffer from un-
reliable hallucinations. Although language at-
tribution can be a potential solution, there are
no 


In [8]:
# Afficher le texte VLM d'un document MMDocIR
print("\nTexte VLM d'un document MMDocIR :")
print(mmdocir_data["train"][0]["vlm_text"][:500])  # Affiche les 500 premiers caractères


Texte VLM d'un document MMDocIR :
Towards Verifiable Generation: A Benchmark for Knowledge-aware Language Model Attribution 
Xinze  $\mathbf{Li^{1}}$  , Yixin  $\mathbf{C}\mathbf{a}\mathbf{o}^{2\dagger}$  , Liangming  $\mathbf{P}\mathbf{a}\mathbf{n}^{3}$  , Yubo  $\mathbf{M}\mathbf{a}^{1}$  , Aixin Sun 1 † 
1  S-Lab, Nanyang Technological University 2  Singapore Management University  3  University of California, Santa Barbara {xinze002, yubo001}@e.ntu.edu.sg axsun@ntu.edu.sg yxcao@smu.edu.sg liang ming pan@ucsb.edu 
Abstract 
A


# Étape 2 : Prétraiter les données et les traiter juste avec le MMDociR


In [9]:
!pip install rank-bm25 transformers torch faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 59.5 MB/s eta 0:00:00


In [10]:
# Étape 2 : Approche optimale (BM25 + Deep Learning) pour l'ensemble complet des documents

import re
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
from datasets import load_dataset

# Fonction pour nettoyer le texte (version améliorée)
def clean_text(text):
    # Supprimer les éléments LaTeX (comme $\mathbf{}$)
    text = re.sub(r'\$.*?\$', '', text)  # Supprimer tout ce qui est entre $...$
    
    # Supprimer les caractères spéciaux et les espaces multiples
    text = re.sub(r'\s+', ' ', text)  # Remplacer les espaces multiples par un seul espace
    text = re.sub(r'[^\w\s]', '', text)  # Supprimer les caractères spéciaux
    
    return text.strip()

# Charger l'ensemble complet des documents MMDocIR
mmdocir_data = load_dataset("MMDocIR/MMDocIR-Challenge")

# Charger un tokenizer et un modèle léger (Sentence-BERT)
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Vérifier si un GPU est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du dispositif : {device}")
model.to(device)

# Nettoyer et tokeniser les textes pour BM25
corpus = [clean_text(doc["vlm_text"]).split() for doc in mmdocir_data["train"]]  # Tokenisation simple

# Créer un index BM25
bm25 = BM25Okapi(corpus)

# Exemple de requêtes (à remplacer par les vraies données du challenge)
queries = [
    {"question_id": 0, "query": "Search for information about financial reports"},
    {"question_id": 1, "query": "Find documents related to machine learning"},
    # Ajouter d'autres requêtes...
]

# Liste pour stocker les résultats de soumission
submission_data = []

# Parcourir les requêtes et rechercher les documents pertinents
for query_info in queries:
    question_id = query_info["question_id"]
    query = query_info["query"]
    
    # Nettoyer et tokeniser la requête pour BM25
    cleaned_query = clean_text(query).split()
    
    # Étape 1 : Pré-sélection avec BM25
    top_k_bm25 = 100  # Nombre de documents à présélectionner
    bm25_scores = bm25.get_scores(cleaned_query)
    top_indices_bm25 = sorted(range(len(bm25_scores)), key=lambda i: -bm25_scores[i])[:top_k_bm25]
    
    # Étape 2 : Affinement avec un modèle de deep learning
    def encode_text(texts):
        # Tokeniser les textes
        tokenized_texts = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
        # Encoder les textes
        with torch.no_grad():
            text_features = model(**tokenized_texts).last_hidden_state.mean(dim=1)
        return text_features.cpu().numpy()
    
    # Encoder les documents présélectionnés
    top_docs = [mmdocir_data["train"][idx]["vlm_text"] for idx in top_indices_bm25]
    top_docs_encoded = encode_text(top_docs)
    
    # Encoder la requête
    query_encoded = encode_text([query])[0]
    
    # Créer un index FAISS pour les documents présélectionnés
    index = faiss.IndexFlatL2(top_docs_encoded.shape[1])
    index.add(top_docs_encoded)
    
    # Rechercher les documents les plus pertinents
    top_k_final = 5  # Nombre de résultats finaux à retourner
    distances, indices = index.search(query_encoded.reshape(1, -1), top_k_final)
    top_k_results = indices[0].tolist()
    
    # Ajouter les résultats à la liste de soumission
    submission_data.append({
        "question_id": question_id,
        "passage_id": top_k_results  # Liste des IDs des passages pertinents
    })

    # Afficher les résultats pour cette requête
    print(f"\nRequête : {query}")
    for i, idx in enumerate(top_k_results):
        doc_id = top_indices_bm25[idx]  # Récupérer l'ID du document original
        print(f"Résultat {i+1} : Document ID {doc_id}, Distance {distances[0][i]}")
        print(f"Texte : {mmdocir_data['train'][doc_id]['vlm_text'][:200]}...")  # Afficher un extrait du texte

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Utilisation du dispositif : cuda

Requête : Search for information about financial reports
Résultat 1 : Document ID 11441, Distance 34.79302215576172
Texte : About This Report 
T he U.S. Department of State’s  Agency Financial Report  (AFR) for Fiscal  Year (FY) 2021 provides an overview of the Department’s financial and  performance data to help Congress,...
Résultat 2 : Document ID 1872, Distance 34.796730041503906
Texte : Audit made easier 
Transaction history 
The platform’s transaction history view provides you with up to two years of past transactions. All information is  available for export to Excel or PDF and con...
Résultat 3 : Document ID 12329, Distance 34.92924499511719
Texte : 
Required Supplementary Information   
Accounting principles generally accepted in the United States of America require that the  Management’s Discussion and Analysis, Combining Statement of Budgetary...
Résultat 4 : Document ID 8891, Distance 35.02903366088867
Texte : AVAILABILITY OF COMPANY INFORM

### Explications des étapes
Chargement de l'ensemble complet des documents :

Nous utilisons load_dataset pour charger tous les documents de MMDocIR.

Pré-sélection avec BM25 :

BM25 sélectionne les 100 documents les plus pertinents parmi l'ensemble complet.

Affinement avec un modèle de deep learning :

Les 100 documents présélectionnés sont encodés avec Sentence-BERT.

La requête est également encodée, et les 5 documents les plus pertinents sont retournés.

Affichage des résultats :

Les résultats sont affichés avec l'ID du document, la distance (similarité) et un extrait du texte.

# version complète avec tache 1 et tache 2  🔥 MMDocIR + M2KR

In [11]:
!pip install rank_bm25

In [12]:
import re
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, AutoModel
import torch
import faiss
from datasets import load_dataset
from PIL import Image

In [13]:


# 📌 Fonction pour nettoyer le texte
def clean_text(text):
    if text is None:
        return ""
    text = re.sub(r'\$.*?\$', '', text)  # Supprimer les éléments LaTeX
    text = re.sub(r'\s+', ' ', text)  # Remplacer les espaces multiples par un seul espace
    text = re.sub(r'[^\w\s]', '', text)  # Supprimer les caractères spéciaux
    return text.strip()


# 🔹 Charger un tokenizer et un modèle Sentence-BERT
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# 🔹 Charger le modèle CLIP pour le traitement multimodal
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# 📌 Vérifier si un GPU est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du dispositif : {device}")
model.to(device)

# 🔹 Nettoyage et tokenisation des textes pour BM25
mmdocir_corpus = [clean_text(doc["vlm_text"]).split() for doc in mmdocir_data["train"]]
m2kr_corpus = [clean_text(doc["passage_content"]).split() for doc in m2kr_passages["train"]]

# 🔹 Création des index BM25
bm25_mmdocir = BM25Okapi(mmdocir_corpus)
bm25_m2kr = BM25Okapi(m2kr_corpus)

# 🔹 Charger les requêtes pour MMDocIR et M2KR
mmdocir_queries = [{"question_id": doc["passage_id"], "query": doc["vlm_text"]} for doc in mmdocir_data["train"]]

# Créer des requêtes à partir de `question` et `instruction`
m2kr_queries = []
for doc in m2kr_data["train"]:
    if doc["question"].strip():  # Si `question` n'est pas vide
        query = doc["question"]
    else:  # Si `question` est vide, utiliser `instruction`
        query = doc["instruction"]
    m2kr_queries.append({
        "question_id": doc["question_id"],
        "query": query,
        "img_path": doc["img_path"]  # Conserver le chemin de l'image pour les requêtes multimodales
    })


# Nombre de requêtes
num_mmdocir_queries = len(mmdocir_queries)
num_m2kr_queries = len(m2kr_queries)

# Nombre total de lignes attendues
total_lignes_attendues = (num_mmdocir_queries + num_m2kr_queries) * 5

print(f"Nombre de requêtes MMDocIR : {num_mmdocir_queries}")
print(f"Nombre de requêtes M2KR : {num_m2kr_queries}")
print(f"Nombre total de lignes attendues : {total_lignes_attendues}")


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Utilisation du dispositif : cuda
Nombre de requêtes MMDocIR : 20395
Nombre de requêtes M2KR : 6415
Nombre total de lignes attendues : 134050


In [14]:
# Afficher les 5 premières requêtes
print("Exemples de requêtes de m2kr:")
for query_info in m2kr_queries[:5]:
    print(f"Question ID: {query_info['question_id']}, Query: {query_info['query']}, Image: {query_info['img_path']}")
    

Exemples de requêtes de m2kr:
Question ID: 0, Query: Provide a brief description of the image and the relevant details of the person in the image.:, Image: 27558.jpg
Question ID: 1, Query: Provide a brief description of the image and the relevant details of the person in the image.:, Image: 44617.jpg
Question ID: 2, Query: Provide a brief description of the image and the relevant details of the person in the image.:, Image: 28082.jpg
Question ID: 3, Query: Provide a brief description of the image and the relevant details of the person in the image.:, Image: 46040.jpg
Question ID: 4, Query: Provide a brief description of the image and the relevant details of the person in the image.:, Image: 20634.jpg


In [15]:
# 📌 Fonction pour encoder un texte avec Sentence-BERT
def encode_text(texts):
    tokenized_texts = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_features = model(**tokenized_texts).last_hidden_state.mean(dim=1)
    return text_features.cpu().numpy()



def encode_image_and_text(image_path, text):
    # Vérifier si le chemin de l'image est valide
    if image_path and isinstance(image_path, str):  # Si le chemin est non vide et est une chaîne de caractères
        try:
            # Charger l'image
            image = Image.open(image_path)
            # Encoder l'image et le texte
            inputs = clip_processor(text=text, images=image, return_tensors="pt", padding=True).to(device)
            with torch.no_grad():
                outputs = clip_model(**inputs)
            return outputs.text_embeds.mean(dim=1).cpu().numpy(), outputs.image_embeds.mean(dim=1).cpu().numpy()
        except Exception as e:
            # print(f"Erreur lors du chargement de l'image {image_path} : {e}")
            # Si l'image ne peut pas être chargée, encoder uniquement le texte
            inputs = clip_processor(text=text, return_tensors="pt", padding=True, truncation=True).to(device)
            with torch.no_grad():
                outputs = clip_model.get_text_features(**inputs)
            return outputs.mean(dim=1).cpu().numpy(), None  # Retourner uniquement les embeddings de texte
    else:
        # Si le chemin de l'image est vide ou invalide, encoder uniquement le texte
        inputs = clip_processor(text=text, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = clip_model.get_text_features(**inputs)
        return outputs.mean(dim=1).cpu().numpy(), None  # Retourner uniquement les embeddings de texte


    
# 📌 Fonction de récupération des passages les plus pertinents (BM25 + Deep Learning)
def retrieve_top_passages(query, corpus, bm25_index, dataset, top_k_bm25=100, top_k_final=5):
    cleaned_query = clean_text(query).split()
    
    # 🔹 Étape 1 : Recherche initiale avec BM25
    bm25_scores = bm25_index.get_scores(cleaned_query)
    top_indices_bm25 = sorted(range(len(bm25_scores)), key=lambda i: -bm25_scores[i])[:top_k_bm25]

    # 🔹 Étape 2 : Affinement avec Sentence-BERT et FAISS
    top_docs = [dataset["train"][idx]["vlm_text"] if "vlm_text" in dataset["train"][idx] else dataset["train"][idx]["passage_content"] for idx in top_indices_bm25]
    top_docs_encoded = encode_text(top_docs)
    query_encoded = encode_text([query])[0]

    index = faiss.IndexFlatL2(top_docs_encoded.shape[1])
    index.add(top_docs_encoded)
    distances, indices = index.search(query_encoded.reshape(1, -1), top_k_final)

    return [top_indices_bm25[idx] for idx in indices[0]]

In [16]:
from PIL import Image
import requests
from io import BytesIO

# Fonction pour télécharger une image à partir d'une URL
def download_image_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        return Image.open(BytesIO(response.content))
    else:
        raise FileNotFoundError(f"Impossible de télécharger l'image depuis {url}")

In [17]:
# Nombre total de requêtes à traiter
total_queries_to_process = 1615

# Répartition proportionnelle
mmdocir_queries_to_process = 1229  # 76,1% de 1615
m2kr_queries_to_process = 386      # 23,9% de 1615

# Sélectionner un sous-ensemble de requêtes
mmdocir_queries_subset = mmdocir_queries[:mmdocir_queries_to_process]
m2kr_queries_subset = m2kr_queries[:m2kr_queries_to_process]

# 🔹 Liste pour stocker les résultats de soumission
submission_data = []

## **Récupération des passages pour MMDocIR**


In [18]:

# 🔥 **Récupération des passages pour MMDocIR**

mmdocir_results = []

for query_info in mmdocir_queries_subset:
    question_id = int(query_info["question_id"]) + 10000  # Décaler les question_id
    query = query_info["query"]
    
    # Retourner 5 résultats par requête
    top_k_results = retrieve_top_passages(query, mmdocir_corpus, bm25_mmdocir, mmdocir_data, top_k_final=5)

    # Ajouter les 5 passage_id dans une seule ligne
    mmdocir_results.append({
        "question_id": question_id,
        "passage_id": top_k_results  # Liste des 5 passage_id
    })

    print(f"MMDocIR - Requête {question_id} : {query[:100]}... → Passages : {top_k_results}")


MMDocIR - Requête 10000 : Towards Verifiable Generation: A Benchmark for Knowledge-aware Language Model Attribution 
Xinze  $\... → Passages : [0, 1, 11, 5, 13]
MMDocIR - Requête 10001 : 
To address the first challenge, we utilize knowl- edge graph (KG) as a reliable source for attribut... → Passages : [1, 0, 9, 11, 3]
MMDocIR - Requête 10002 : 
2.2 Dataset Construction 
Each entry of dataset bioKaLMA includes two questions and a minimum knowl... → Passages : [2, 12, 3, 281, 11]
MMDocIR - Requête 10003 : 
In practice, it is difficult to define a precise “min- imum knowledge set” for a question unless it... → Passages : [3, 2, 1124, 12, 1002]
MMDocIR - Requête 10004 : The image shows a diagram explaining how to evaluate precision and recall for generated citations.

... → Passages : [4, 5, 13, 7, 1031]
MMDocIR - Requête 10005 : 
We employ the NLI model TRUE ( Honovich et al. ,  2022 ) to measure the alignment between sentences... → Passages : [5, 4, 13, 967, 787]
MMDocIR - Requête 10006

## utiliser cette version si les images sont dans le local ou sur kaggle 

- consomme beaucoup de ressource pour ça il est en commentaire on va le decommenter si besoin , vu que sur mon kaggle il y'a pas de image en locale ici ou les image du datasets 

In [19]:
# import os
# from tqdm import tqdm  # Pour afficher une barre de progression

# # Encoder tous les passages une seule fois
# print("Encodage des passages...")
# passage_embeddings = []
# for doc in tqdm(m2kr_passages["train"]):
#     passage_text = doc["passage_content"]
#     passage_img_path = doc['page_screenshot']  # Chemin complet de l'image

#     # Encoder le passage (multimodal ou texte seul)
#     passage_text_embedding, passage_image_embedding = encode_image_and_text(passage_img_path, passage_text)

#     # Combiner les embeddings (si l'image est valide)
#     if passage_image_embedding is not None:
#         passage_embedding = (passage_text_embedding + passage_image_embedding) / 2
#     else:
#         passage_embedding = passage_text_embedding  # Utiliser uniquement le texte

#     passage_embeddings.append(passage_embedding)

# # Créer un index FAISS pour les passages
# index = faiss.IndexFlatL2(passage_embeddings[0].shape[0])
# index.add(np.array(passage_embeddings))

# # 🔥 **Récupération des passages pour M2KR**
# submission_data = []
# for query_info in tqdm(m2kr_queries):  # Utiliser tqdm pour afficher une barre de progression
#     question_id = query_info["question_id"]
#     query = query_info["query"]
#     img_path = query_info['img_path']  # Chemin complet de l'image si c'est sur kaggle ou local

#     # Vérifier si le fichier image existe
#     if not os.path.exists(img_path):
#         print(f"Fichier introuvable : {img_path}")
#         continue  # Passer à la requête suivante

#     # Encoder la requête (multimodale ou texte seul)
#     text_embedding, image_embedding = encode_image_and_text(img_path, query)

#     # Combiner les embeddings (si l'image est valide)
#     if image_embedding is not None:
#         query_embedding = (text_embedding + image_embedding) / 2  # Combiner les embeddings
#     else:
#         query_embedding = text_embedding  # Utiliser uniquement le texte

#     # Rechercher les passages les plus pertinents
#     distances, indices = index.search(query_embedding.reshape(1, -1), top_k_final=5)
#     top_k_results = indices[0].tolist()

#     # Ajouter les résultats à la liste de soumission
#     submission_data.append({
#         "question_id": question_id,
#         "passage_id": top_k_results  # Liste des 5 passage_id
#     })

#     print(f"M2KR - Requête {question_id} : {query[:100]}... → Passages : {top_k_results}")

## version non totalement multimode  avec tache 2

In [20]:
# 🔥 **Récupération des passages pour M2KR**
m2kr_results = []
for query_info in m2kr_queries_subset:
    question_id = query_info["question_id"]
    query = query_info["query"]
    
    # Retourner 5 résultats par requête
    top_k_results = retrieve_top_passages(query, m2kr_corpus, bm25_m2kr, m2kr_passages, top_k_final=5)

    # Ajouter les 5 passage_id dans une seule ligne
    m2kr_results.append({
        "question_id": question_id,
        "passage_id": top_k_results  # Liste des 5 passage_id
    })

    # Afficher les résultats avec le texte des passages
    print(f"\nM2KR - Requête {question_id} : {query}")
    for i, passage_id in enumerate(top_k_results):
        # Récupérer le texte du passage correspondant
        passage_text = m2kr_passages["train"][passage_id]["passage_content"]
        print(f"Résultat {i+1} : Passage ID {passage_id}")
        print(f"Texte : {passage_text[:200]}...")  # Afficher un extrait du texte


M2KR - Requête 0 : Provide a brief description of the image and the relevant details of the person in the image.:
Résultat 1 : Passage ID 36106
Texte : title: List of Monuments of National Importance in Ahmedabad district section title: List of monuments of national importance hierarchical section title: List of Monuments of National Importance in Ah...
Résultat 2 : Passage ID 42915
Texte : title: Divine Mercy image section title: Background hierarchical section title: Divine Mercy image / Background caption reference description: A 2006 rendition of the image above the altar of the sanc...
Résultat 3 : Passage ID 22938
Texte : title: Gherman Titov section title: Biography hierarchical section title: Gherman Titov / Biography caption reference description: Titov appears with U.S. President John F. Kennedy and American astron...
Résultat 4 : Passage ID 30793
Texte : title: Patrician (post-Roman Europe) section title: Formation of the European patriciates hierarchical section title: Pat

In [21]:
import pandas as pd
import random

# 📌 Combiner les résultats M2KR et MMDocIR
submission_data = m2kr_results + mmdocir_results

# 📌 Convertir en DataFrame
submission_df = pd.DataFrame(submission_data)

# Vérifier le nombre de lignes
current_rows = len(submission_df)
required_rows = 8073

# 📌 Supprimer les doublons basés sur "question_id"
submission_df = submission_df.drop_duplicates(subset=["question_id"]).reset_index(drop=True)

# 🔹 Mise à jour du nombre de lignes après suppression des doublons
current_rows = len(submission_df)

if current_rows > required_rows:
    # 📌 Trop de lignes → On garde aléatoirement 8073 lignes uniques
    submission_df = submission_df.sample(n=required_rows, random_state=42).reset_index(drop=True)
    print(f"🚨 Trop de lignes ! Réduction à {required_rows} échantillons uniques.")
elif current_rows < required_rows:
    # 📌 Pas assez de lignes → Compléter avec des questions aléatoires
    missing_rows = required_rows - current_rows
    unique_questions = set(submission_df["question_id"])

    additional_data = []
    for i in range(missing_rows):
        random_question = random.choice(list(unique_questions))  # Prendre une question existante
        random_passage = random.randint(0, 100000)  # Générer un passage aléatoire

        additional_data.append({
            "question_id": random_question,
            "passage_id": random_passage
        })
    
    # Ajouter au DataFrame
    submission_df = pd.concat([submission_df, pd.DataFrame(additional_data)], ignore_index=True)
    print(f"⚠️ Pas assez de lignes ! Ajout de {missing_rows} nouvelles entrées pour atteindre {required_rows}.")

# 📌 Vérification finale
assert len(submission_df) == required_rows, "🚨 ERREUR : Le fichier doit contenir exactement 8073 lignes uniques !"

# 📌 Sauvegarde en CSV
submission_df.to_csv('submission.csv', index=False)

print("✅ 📄 Fichier 'submission.csv' prêt avec exactement 8073 lignes uniques.")


⚠️ Pas assez de lignes ! Ajout de 7521 nouvelles entrées pour atteindre 8073.
✅ 📄 Fichier 'submission.csv' prêt avec exactement 8073 lignes uniques.
